<a href="https://colab.research.google.com/github/Akashh127/Churning_Model-using-ANN/blob/main/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**ANN IMPLEMENTAION**

In [2]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [3]:
#import basic libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
#reading the dataset
dataset=pd.read_csv('/content/drive/MyDrive/Churn_Modelling (1)/Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


The column named from 'RowNumber' to 'EstimatedSalary' is denoted as independent features but we take notice from 'CreditScore' to 'EstimatedSalary' because the feature such as 'RowNumber','CustomerId' and 'surname' doesn't make any sense.
The column named Exited is denoted as dependent feature.

In [6]:
#divide the dataset into independent and dependant features
X=dataset.iloc[:,3:13]   #index locations
y=dataset.iloc[:,13]

In [9]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


The dataset is not that clean because we do have non-numerial and categorical column type of data.so we need to fix this categorical features inorder to revmove it we use 'getdummies'in pandas or one-hot encoding.

In [8]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

Handling the catgorical features of independent variable.

In [11]:
pd.get_dummies(X["Geography"])

,France,Germany,Spain
0,1,0,0
1,0,0,1
2,1,0,0
3,1,0,0
4,0,0,1
...,...,...,...
9995,1,0,0
9996,1,0,0
9997,1,0,0
9998,0,1,0


The parameter 'drop_first=true' is giving us two columns instead of 3 columns so we can represent the feature by using two columns rather 3 columns

In [10]:
#Feature engineering
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)

In [12]:
#Concatenate the Data Frames
X=X.drop(['Geography','Gender'],axis=1) #axis is used to drop the columns not rows


In [13]:
X=pd.concat([X,geography,gender],axis=1)

In [14]:
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


In [15]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [16]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler #also min & max scaler is there
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [17]:
X_train

array([[ 0.16958176, -0.46460796,  0.00666099, ..., -0.5698444 ,
         1.74309049, -1.09168714],
       [-2.30455945,  0.30102557, -1.37744033, ...,  1.75486502,
        -0.57369368,  0.91601335],
       [-1.19119591, -0.94312892, -1.031415  , ..., -0.5698444 ,
        -0.57369368, -1.09168714],
       ...,
       [ 0.9015152 , -0.36890377,  0.00666099, ..., -0.5698444 ,
        -0.57369368,  0.91601335],
       [-0.62420521, -0.08179119,  1.39076231, ..., -0.5698444 ,
         1.74309049, -1.09168714],
       [-0.28401079,  0.87525072, -1.37744033, ...,  1.75486502,
        -0.57369368, -1.09168714]])

In [18]:
y_train

7389    0
9275    0
2995    0
5316    0
356     0
       ..
9225    0
4859    0
3264    0
9845    0
2732    1
Name: Exited, Length: 8000, dtype: int64

In [19]:
X_train.shape

(8000, 11)

Creation and Training of the Artifical Neutal Network by using tensorflow library.
Tensorflow:The open-source library by google deep-mines team with keras intergrated after the version of 2.0.

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU,PReLU,ELU
from tensorflow.keras.layers import Dropout

In [21]:
# Initialising the ANN
classifier = Sequential()
# Adding the input layer
classifier.add(Dense(units=11,activation='relu'))
# Adding the first hidden layer
classifier.add(Dense(units=7,activation='relu'))
# Adding the second hidden layer
classifier.add(Dense(units=6,activation='relu'))
# Adding the output layer
classifier.add(Dense(units=1,activation='sigmoid'))


you can also modify the ooptimizer also:
import tensorflow
opt=tensorflow.feras.optimizers.Adam(learning_rate=0.01)
# change optimizer=opt

In [22]:
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

Early stopping:Stop training when a monitored metric has stopped improving.The callback is the function to assign the early stopping.
link-https://keras.io/api/callbacks/early_stopping/

In [23]:
#Early stopping
import tensorflow as tf

early_stopping=tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.0001,
    patience=20,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

In [24]:
model_history=classifier.fit(X_train,y_train,validation_split=0.33,batch_size=10,epochs=1000,callbacks=early_stopping)

Epoch 1/1000
536/536 [==============================] - 7s 7ms/step - loss: 0.5038 - accuracy: 0.7962 - val_loss: 0.4638 - val_accuracy: 0.7955
Epoch 2/1000
536/536 [==============================] - 2s 4ms/step - loss: 0.4464 - accuracy: 0.7962 - val_loss: 0.4470 - val_accuracy: 0.7955
Epoch 3/1000
536/536 [==============================] - 2s 4ms/step - loss: 0.4316 - accuracy: 0.7962 - val_loss: 0.4409 - val_accuracy: 0.7955
Epoch 4/1000
536/536 [==============================] - 2s 4ms/step - loss: 0.4261 - accuracy: 0.7962 - val_loss: 0.4362 - val_accuracy: 0.7955
Epoch 5/1000
536/536 [==============================] - 2s 4ms/step - loss: 0.4202 - accuracy: 0.8029 - val_loss: 0.4305 - val_accuracy: 0.8054
Epoch 6/1000
536/536 [==============================] - 3s 6ms/step - loss: 0.4122 - accuracy: 0.8151 - val_loss: 0.4223 - val_accuracy: 0.8088
Epoch 7/1000
536/536 [==============================] - 2s 4ms/step - loss: 0.4026 - accuracy: 0.8266 - val_loss: 0.4135 - val_accuracy:

In [25]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

63/63 [==============================] - 0s 1ms/step


In [26]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1502,   93],
       [ 196,  209]])

In [27]:
# Calculate the Accuracy
from sklearn.metrics import accuracy_score
score=accuracy_score(y_pred,y_test)

In [28]:
score

0.8555

In [31]:
classifier.get_weights()

[array([[ 0.12508646, -0.83332425,  0.05439235, -0.04339939, -0.06251334,
         -0.01148246,  0.02437222,  0.36884385,  0.06119503,  0.27811083,
         -0.6272973 ],
        [ 0.06757116, -0.5667826 ,  1.1989797 ,  0.30049762,  0.35831335,
          0.00930513,  0.33503538, -0.03805084,  0.34889138, -0.9021179 ,
          0.18292937],
        [ 0.18422692, -0.0560028 ,  0.2803828 ,  0.03021228, -0.09231378,
         -0.2106357 , -0.17604204,  0.23036699, -0.20722318, -0.25348005,
          0.5351945 ],
        [-0.00936315,  0.34929743, -0.46087596,  0.14135334, -0.3941778 ,
         -0.05228113,  0.6040631 , -0.6742752 ,  0.14156465,  0.08896723,
         -0.2286723 ],
        [ 0.05513705, -0.01608134,  0.1289922 ,  1.3015783 , -1.1790901 ,
          0.19396588,  0.54970413, -0.00677497,  0.42054442,  0.07965612,
          0.4523911 ],
        [-0.01543958, -0.32859495, -0.21031305, -0.01977932, -0.05131758,
          0.0081913 ,  0.07682771, -0.25153157,  0.4119589 , -0.1139227